# Jane Street Market Prediction - Submission
This is the actual notebook submitted to the competition. First, we load the trained models and use them to implement the `predict()` function.

In [ ]:
import os

import pandas as pd

# load models
json_file = os.path.join(os.pardir, "input",
                         "game-of-life-training",
                         "model.json")
weight_file = os.path.join(os.pardir, "input",
                           "game-of-life-training",
                           "model.h5")

with open(json_file, "r") as f:
    model_json = f.read()

custom_objects = {"TorusPadding": TorusPadding}
with keras.utils.custom_object_scope(custom_objects):    
    raw_model = keras.models.model_from_json(model_json)
    raw_model.load_weights(weight_file)

# add a rounding layer to output 0s and 1s
inputs = layers.Input(shape=[25, 25, 1], name="inputs")
raw_pred = raw_model(inputs)
outputs = layers.Activation(tf.round)(raw_pred)

model = keras.Model(inputs=inputs, outputs=outputs, name="inference")
model.compile()



# tune thresholds for highest utility score on training set...
# https://www.kaggle.com/gogo827jz/jane-street-super-fast-utility-score-function/
def utility_score_pd(date, weight, resp, action):
    count_i = len(pd.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u


# function to return predictions for the current trade
# in test_df, using historical data in hist_df
def predict(test_df, hist_df):
    return 0

Then we use the Jane Street API to make the predictions. In this setup the iterator `iter_test` yields single samples `test_df` in chronological order. The predict function needs to decide whether to accept or reject the trade at this time-point using only the current sample and the previous samples stored in `hist_df`. Set `pred_df` to 1 to accept the trade or to 0 to reject the trade.

In [ ]:
import janestreet

env = janestreet.make_env()
iter_test = env.iter_test()

hist_df = pd.DataFrame()

for (test_df, pred_df) in iter_test:
    # accept/reject current trade
    pred_df.action = predict(test_df, hist_df)
    env.predict(sample_prediction_df)
    
    # store the new sample in the history data frame
    hist_df.append(test_df)